# Memory

In [19]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.language_models import FakeListChatModel
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.messages import trim_messages, HumanMessage


class PrintOutputCallback(BaseCallbackHandler):
    def on_chat_model_start(self, serialized, messages, **kwargs):
        print(f"Amount of input messages: {len(messages)}")


sessions = {}
handler = PrintOutputCallback()
llm = FakeListChatModel(responses=["ai1", "ai2", "ai3"])

def get_session_history(session_id: str):
    if session_id not in sessions:
        sessions[session_id] = InMemoryChatMessageHistory()
    return sessions[session_id]

trimmer = trim_messages(
    max_tokens=1,
    strategy="last",
    token_counter=len,
    include_system=True,
    start_on="human",
)

raw_chain = trimmer | llm
chain = RunnableWithMessageHistory(raw_chain, get_session_history)

config = {"callbacks": [PrintOutputCallback()], "configurable": {"session_id": "1"}}
_ = chain.invoke(
    [HumanMessage("Hi!")],
    config=config,
)
print(f"History length: {len(sessions['1'].messages)}")

_ = chain.invoke(
    [HumanMessage("How are you?")],
    config=config,
)
print(f"History length: {len(sessions['1'].messages)}")

Amount of input messages: 1
History length: 2
Amount of input messages: 1
History length: 4


In [20]:
sessions["1"].messages

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai1', additional_kwargs={}, response_metadata={}, id='run-61ec6b0b-6631-48c7-95b6-61da8ec7aca7-0'),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ai2', additional_kwargs={}, response_metadata={}, id='run-387c2f6c-209d-4de3-8cc4-fdb4afa88fe5-0')]

In [21]:
trimmer.invoke(sessions["1"].messages)

[]

In [22]:
from langgraph.graph import MessageGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage
from langgraph.graph import START, END


def test_node(state):
   # ignore the last message since it's an input one
   print(f"History length = {len(state[:-1])}")
   return [AIMessage(content="Hello!")]

builder = MessageGraph()
builder.add_node("test_node", test_node)
builder.add_edge(START, "test_node")
builder.add_edge("test_node", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [23]:
_ = graph.invoke([HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}})
_ = graph.invoke([HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-b"}})
_ = graph.invoke([HumanMessage(content="test")], config={"configurable": {"thread_id": "thread-a"}})

History length = 0
History length = 0
History length = 2


In [24]:
checkpoints = list(memory.list(config={"configurable": {"thread_id": "thread-a"}}))
for check_point in checkpoints:
  print(check_point.config["configurable"]["checkpoint_id"])

1eff8dbc-fe59-662a-8004-064ebb341150
1eff8dbc-fe58-6450-8003-2ee2b7ffac5e
1eff8dbc-fe56-68b2-8002-c5562269bb46
1eff8dbc-fe48-6f50-8001-710b65f78d1f
1eff8dbc-fe3b-6864-8000-d5f811d92d2f
1eff8dbc-fe29-6768-bfff-2f0d41677e7c


In [25]:
checkpoint_id = checkpoints[-1].config["configurable"]["checkpoint_id"]
_ = graph.invoke(
    [HumanMessage(content="test")],
    config={"configurable": {"thread_id": "thread-a", "checkpoint_id": checkpoint_id}})

History length = 0


In [26]:
checkpoint_id = checkpoints[-3].config["configurable"]["checkpoint_id"]
_ = graph.invoke(
    [HumanMessage(content="test")],
    config={"configurable": {"thread_id": "thread-a", "checkpoint_id": checkpoint_id}})

History length = 2
